test examples: https://github.com/Azure/CSContainers/blob/master/Project-Archon/private-preview/Shelf-Analysis/cpg-privatepreview-api-tutorial.md 

In [73]:
import os 
from urllib.parse import urljoin

In [74]:
api_endpoint = os.environ.get('api_endpoint', 'https://<region>.api.cognitive.microsoft.com/')
api_key = os.environ.get('api_key', '12233344445555566666677777779999')


api_datasets = '/vision/v4.0-preview.1/datasets'
api_imagenalysis = '/vision/v4.0-preview.1/operations/shelfanalysis-productunderstanding:analyze'
api_imagerectify = '/vision/v4.0-preview.1/operations/shelfanalysis-productunderstanding:rectify'


stitched_image = 'https://godzillasinastorage.blob.core.windows.net/cv4/stitched-rectified2.png'
long_shelve = 'https://godzillasinastorage.blob.core.windows.net/cv4/long_shelve.jpg'

In [75]:
# image analysis 
# https://github.com/Azure/CSContainers/blob/master/Project-Archon/private-preview/Shelf-Analysis/cpg-privatepreview-apis-operations.md#v40-preview1operationsshelfanalysis-productunderstandinganalyze

import requests
request = {
             "url": stitched_image
}

api_endpoint = 'https://eastus.api.cognitive.microsoft.com/vision/v4.0-preview.1/operations/shelfanalysis-productunderstanding:analyze'
# post request to imageanaylsis
response = requests.post(urljoin(api_endpoint,api_imagenalysis), headers={'Ocp-Apim-Subscription-Key': api_key}, json=request)


In [76]:
print(response.content)

b'{"imageMetadata":{"width":2063,"height":1538},"products":[{"id":"1","boundingBox":{"x":1518,"y":1235,"w":144,"h":162},"classifications":[{"confidence":0.9701688,"label":"product"}]},{"id":"2","boundingBox":{"x":303,"y":1263,"w":147,"h":139},"classifications":[{"confidence":0.96984136,"label":"product"}]},{"id":"3","boundingBox":{"x":101,"y":339,"w":79,"h":150},"classifications":[{"confidence":0.96909773,"label":"product"}]},{"id":"4","boundingBox":{"x":1663,"y":1020,"w":81,"h":167},"classifications":[{"confidence":0.9688566,"label":"product"}]},{"id":"5","boundingBox":{"x":429,"y":1056,"w":150,"h":139},"classifications":[{"confidence":0.9686272,"label":"product"}]},{"id":"6","boundingBox":{"x":1038,"y":565,"w":88,"h":141},"classifications":[{"confidence":0.9665815,"label":"product"}]},{"id":"7","boundingBox":{"x":254,"y":334,"w":81,"h":151},"classifications":[{"confidence":0.9647248,"label":"product"}]},{"id":"8","boundingBox":{"x":671,"y":323,"w":83,"h":153},"classifications":[{"con

In [77]:
import json

#print formatted json
# print(json.dumps(response.json(), indent=2))

img_boxes = response.json()
img_boxes.keys()


dict_keys(['imageMetadata', 'products', 'gaps'])

In [78]:
img_metadata = img_boxes.get('imageMetadata', [])
img_products = img_boxes.get('products', [])
img_gaps = img_boxes.get('gaps', [])

In [79]:
print(len(img_gaps))
print(len(img_products))

# print(img_gaps[0])

bb_gaps = [ [(bb['boundingBox']['x'], bb['boundingBox']['y']), 
   (bb['boundingBox']['x']+bb['boundingBox']['w'], bb['boundingBox']['y']+bb['boundingBox']['h'])]
 for bb in img_gaps]

8
126


In [80]:
# load image from url
from PIL import Image, ImageDraw
from io import BytesIO
import requests

img_download = requests.get(request['url'])

print(type(img_download.content))
print(BytesIO(img_download.content))

# load bytes into image
img = Image.open(BytesIO(img_download.content), 'r')


<class 'bytes'>


In [81]:
# draw rectangles from bb_gaps
for bb in bb_gaps:
    ImageDraw.Draw(img).rectangle(bb, outline="red", fill="grey", width=5)
    #write text ofsett from top left corner, 20px, white text, red background using arial font
    ImageDraw.Draw(img).text(
        (bb[0][0]+10, bb[0][1]+5), 
        'Gap', 
        fill="red",
        )    
img.show()

In [82]:
# # draw rectangles from bb_products in green
# bb_products = [ [(bb['boundingBox']['x'], bb['boundingBox']['y']),
#     (bb['boundingBox']['x']+bb['boundingBox']['w'], bb['boundingBox']['y']+bb['boundingBox']['h'])]
#     for bb in img_products]

# for bb in bb_products:
#     ImageDraw.Draw(img).rectangle(bb, outline="green", width=5)
# img.show()


# Summarisation

In [83]:
print(img_products[0])

# convert to a dataframe
import pandas as pd
df = pd.DataFrame(img_products)
df.head()


{'id': '1', 'boundingBox': {'x': 1518, 'y': 1235, 'w': 144, 'h': 162}, 'classifications': [{'confidence': 0.9701688, 'label': 'product'}]}


,id,boundingBox,classifications
0,1,"{'x': 1518, 'y': 1235, 'w': 144, 'h': 162}","[{'confidence': 0.9701688, 'label': 'product'}]"
1,2,"{'x': 303, 'y': 1263, 'w': 147, 'h': 139}","[{'confidence': 0.96984136, 'label': 'product'}]"
2,3,"{'x': 101, 'y': 339, 'w': 79, 'h': 150}","[{'confidence': 0.96909773, 'label': 'product'}]"
3,4,"{'x': 1663, 'y': 1020, 'w': 81, 'h': 167}","[{'confidence': 0.9688566, 'label': 'product'}]"
4,5,"{'x': 429, 'y': 1056, 'w': 150, 'h': 139}","[{'confidence': 0.9686272, 'label': 'product'}]"
